# Imports & Data Loading

In [3]:
import numpy as np
import pandas as pd

from functools import reduce

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [4]:
df_2013 = pd.read_csv('../data/2013.csv')
df_2014 = pd.read_csv('../data/2014.csv')
df_2015 = pd.read_csv('../data/2015.csv')
df_2016 = pd.read_csv('../data/2016.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2018 = pd.read_csv('../data/2018.csv')
df_2019 = pd.read_csv('../data/2019.csv')
df_2020 = pd.read_csv('../data/2020.csv')
df_2021 = pd.read_csv('../data/2021.csv')
df_2022 = pd.read_csv('../data/2022.csv')


dictionary_df = {
    '2013': df_2013,
    '2014': df_2014,
    '2015': df_2015,
    '2016': df_2016,
    '2017': df_2017,
    '2018': df_2018,
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
}

for key, df in dictionary_df.items():
    dictionary_df[key] = df[(df['Donor Agency'] == 'Bundesministerium fnr Wirtschaftliche Zusammenarbeit und Entwicklung')]
    dictionary_df[key] = dictionary_df[key][dictionary_df[key]['Sector'] != 'Administrative Costs of Donors']
    
    

# Data Processing for Globe Stacked

In [5]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



## Combined

In [8]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)


# globe_df = globe_df.loc[[38]]

In [9]:
globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
0,229.878097,19.054332,286.807860,31.714736,197.429500,33.457037,346.234112,38.308328,311.670340,42.762633,255.815158,41.988156,241.099156,40.556326,196.114075,34.977410,352.840862,51.961490,201.939382,16.806849
1,132.706037,34.375215,129.848523,17.713145,182.614727,79.600243,129.000894,21.123321,240.258759,109.944048,262.731752,63.757281,555.037032,147.663336,686.808006,180.211271,643.229422,136.808525,627.644704,195.336748
2,19.317268,5.281618,16.371641,5.216575,19.155382,6.609535,18.968980,6.652384,21.017326,7.728618,55.514401,44.503031,19.355900,5.719183,30.080577,7.164332,29.366152,10.070879,25.328362,9.591473
3,6.080655,2.350579,5.218615,4.723721,6.014444,5.011871,4.684949,3.522966,5.347355,4.006726,6.986998,5.201791,9.983005,6.724355,11.714897,5.986887,25.777176,7.808470,16.273651,8.764504
4,61.039902,16.707576,73.783053,35.047207,48.050705,6.148801,77.236341,29.640753,95.230412,58.889436,59.652114,26.355707,103.276123,56.378290,146.314609,103.204088,82.869791,34.533597,107.665968,67.062139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.232934,3.934650,27.979557,9.016008,31.071347,2.476874,31.393523,3.293854
139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.435574,4.101530,5.156201,2.822914,7.093164,3.505433,7.352916,3.160977
140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.828161,7.896584,34.419681,9.418930,29.927608,9.239338,37.365181,18.475603
141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.119008,0.000000,15.884541,2.771182


In [232]:
years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - globe_df[clim_rel_amount_col]

# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_rel_amount_{year}' for year in years] + 
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')



# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(lambda x: 'Climate Finance' if 'clim_rel_amount' in x else 'Other Funds')

melted_df['Amount'] = melted_df['Amount'] * 1_000_000

replacements = {
    'Climate Finance': 'Klimafinanzierung',
    'Other Funds': 'Andere ODA'
}
melted_df['Type'] = melted_df['Type'].replace(replacements)

melted_df.to_csv("../upload_data/globe_df.csv")

## Split

In [177]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')\
                   or col.startswith('clim_adapt_') or col.startswith('clim_miti_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)
globe_df = globe_df.loc[[142]]


years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_adapt_col = f'clim_adapt_amount_{year}'
    clim_miti_col = f'clim_miti_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - (globe_df[clim_adapt_col] + globe_df[clim_miti_col])


# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_adapt_amount_{year}' for year in years] +
                          [f'clim_miti_amount_{year}' for year in years] +
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')


# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(
    lambda x: 'Klimaanpassung Finanzierung' if 'clim_adapt_amount' in x 
    else ('Klimaschutz Finanzierung' if 'clim_miti_amount' in x 
    else 'Andere ODA')
)
melted_df['Amount'] = melted_df['Amounbt'] * 1_000_000




In [178]:
melted_df.to_csv("../upload_data/split_globe.csv")

read_df = pd.read_csv('../upload_data/split_globe.csv')


In [179]:
split_df = pd.read_csv('../upload_data/split_globe.csv')


fig = px.bar(split_df, x='Year', y='Amount', color='Type',
            title='Globale Finanzierungssummen',
            labels={'Amount': 'Finanzierungssumme ($)', 'Year': 'Jahr'},
            category_orders={'Type': ['Andere ODA','Klimaschutz Finanzierung', 'Klimaanpassung Finanzierung']},
            color_discrete_map={'Andere ODA': 'orange', 'Klimaschutz Finanzierung': 'green', 'Klimaanpassung Finanzierung': 'blue'}
            )# This ensures consistent color ordering

fig.update_layout(title_x=0.5)

fig.show()

# Waterfall

In [10]:
globe_df = globe_df.loc[[142]]

In [11]:
globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
142,5115.784928,992.115751,6323.119826,1197.406245,5316.503434,1152.733316,6897.008022,1660.725242,7204.68697,1715.085188,8012.691583,1904.175131,8632.582927,2310.213481,10294.707282,2563.918782,10593.415129,2972.043199,11452.254162,3333.997495


In [12]:
years = [col.split('_')[-1] for col in globe_df.columns if col.startswith('amount_')]

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    clim_rel_percent_col = f'clim_rel_percent_{year}'
    
    # Calculate the clim_rel_percent column for the year
    # Note: It's a good practice to handle division by zero or check if 'amount_' is not zero to avoid ZeroDivisionError.
    globe_df[clim_rel_percent_col] = (globe_df[clim_rel_amount_col] / globe_df[amount_col]) * 100
    
    

selected_columns = [col for col in globe_df.columns if col.startswith('clim_rel_percent_')]
filtered_df = globe_df[selected_columns]


# Melt the DataFrame to get it in the right shape for plotting
melted_df = filtered_df.reset_index().melt(id_vars=['index'], var_name='Year', value_name='Percentage')

# Clean the 'Year' column to keep only the year part
melted_df['Year'] = melted_df['Year'].str.replace('clim_rel_percent_', '')

melted_df = melted_df.drop(columns = 'index')

melted_df['Change'] = melted_df['Percentage'].diff()

melted_df

,Year,Percentage,Change
0,2013,19.393226,NaN
1,2014,18.936953,-0.456273
2,2015,21.682170,2.745217
3,2016,24.078923,2.396753
4,2017,23.805131,-0.273792
5,2018,23.764488,-0.040643
6,2019,26.761556,2.997068
7,2020,24.905213,-1.856343
8,2021,28.055572,3.150359
9,2022,29.112151,1.056579


In [13]:
melted_df.to_csv('../upload_data/globe_waterfall.csv')

# globe_df_to_show

In [18]:
globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,...,clim_rel_percent_2013,clim_rel_percent_2014,clim_rel_percent_2015,clim_rel_percent_2016,clim_rel_percent_2017,clim_rel_percent_2018,clim_rel_percent_2019,clim_rel_percent_2020,clim_rel_percent_2021,clim_rel_percent_2022
142,5115.784928,992.115751,6323.119826,1197.406245,5316.503434,1152.733316,6897.008022,1660.725242,7204.68697,1715.085188,...,19.393226,18.936953,21.68217,24.078923,23.805131,23.764488,26.761556,24.905213,28.055572,29.112151


In [35]:
df_melted = pd.melt(globe_df, var_name='variable', value_name='value')

# Extract year and type
df_melted['year'] = df_melted['variable'].str.extract(r'(\d+)$')[0].astype(int)
df_melted['type'] = df_melted['variable'].str.extract(r'^(.*?)_\d+$')[0]

# Pivot table to restructure into the desired format
df_final = df_melted.pivot_table(index='year', columns='type', values='value', aggfunc='first').reset_index()

# Rename columns as needed
df_final.columns.name = None  # Remove the pivot table multi-level indexing name

df_final = df_final.rename(columns= {'amount': 'Total Finance', 
                          'clim_rel_amount': 'Climate Relevant Sum',
                          'clim_rel_percent': 'Climate Relevant Percentage', 'year':'Year'
                         }).set_index('Year')

In [36]:
df_final.to_csv('../upload_data/globe_df_to_show.csv')

In [37]:
pd.read_csv('../upload_data/globe_df_to_show.csv')

,Year,Total Finance,Climate Relevant Sum,Climate Relevant Percentage
0,2013,5115.784928,992.115751,19.393226
1,2014,6323.119826,1197.406245,18.936953
2,2015,5316.503434,1152.733316,21.682170
3,2016,6897.008022,1660.725242,24.078923
4,2017,7204.686970,1715.085188,23.805131
5,2018,8012.691583,1904.175131,23.764488
6,2019,8632.582927,2310.213481,26.761556
7,2020,10294.707282,2563.918782,24.905213
8,2021,10593.415129,2972.043199,28.055572
9,2022,11452.254162,3333.997495,29.112151


# all_country_df.csv

In [38]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)


df_merged.to_csv("../upload_data/all_country_df.csv")


# Data Processing For Sector Breakdown

In [203]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation', 'Sector']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby('Sector').sum()[['Amount','clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['non_miti_adapt'] =grouped_df['Amount'] - (grouped_df['clim_miti_amount'] + grouped_df['clim_adapt_amount'])
    grouped_df['non_clim'] =grouped_df['Amount'] - grouped_df['clim_rel_amount']
    
        
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'non_miti_adapt': f"non_miti_adapt_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'non_clim': f"non_clim_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    



In [204]:
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Sector', how='outer'), processed_grouped_dfs)


In [205]:
df_merged.to_csv('../upload_data/sector_analysis.csv')

df_merged = pd.read_csv('../upload_data/sector_analysis.csv')

In [218]:
list(range(2013,2023))

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [206]:
year = 2013

selected_year = [col for col in df_merged.columns if col.endswith(f'{year}')]
selected_year.append('Sector')
filtered_year_df = df_merged[selected_year]

In [209]:
clim_rel_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_rel_amount_{year}', title='Climate Related Amount by Sector')
clim_rel_fig.show()


In [213]:
non_clim_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'non_clim_{year}', title='Non Climate Related Amount by Sector')
non_clim_fig.show()


In [214]:
clim_adapt_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_adapt_amount_{year}', title='Climate Apaptation Amount by Sector')
clim_adapt_fig.show()


In [215]:
clim_miti_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_miti_amount_{year}', title='Climate Mitigation Amount by Sector')
clim_miti_fig.show()


## By Country

In [235]:
use_df

,Recipient Name,Amount,Climate Mitigation,Climate Adaptation,Sector,clim_rel,clim_rel_amount,clim_adapt_amount,clim_miti_amount
276,Egypt,0.807941,0.0,0.0,IV.1. General Environment Protection,0.0,0.000000,0.0,0.000000
277,Egypt,0.041383,0.0,0.0,IV.1. General Environment Protection,0.0,0.000000,0.0,0.000000
278,Egypt,2.600523,1.0,0.0,IV.1. General Environment Protection,0.5,1.300261,0.0,1.300261
280,Egypt,0.217925,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
317,Egypt,0.021100,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
12164,Syrian Arab Republic,0.011091,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
12165,West Bank and Gaza Strip,0.055108,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
12173,Yemen,0.010539,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000
12174,"Middle East, regional",0.029859,0.0,0.0,IV.2. Other Multisector,0.0,0.000000,0.0,0.000000


In [249]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation', 'Sector']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Sector', 'Recipient Name']).sum()[['Amount','clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['non_miti_adapt'] =grouped_df['Amount'] - (grouped_df['clim_miti_amount'] + grouped_df['clim_adapt_amount'])
    grouped_df['non_clim'] =grouped_df['Amount'] - grouped_df['clim_rel_amount']
    
        
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'non_miti_adapt': f"non_miti_adapt_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'non_clim': f"non_clim_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    



In [251]:
processed_grouped_dfs[0]

amount_2013  \
Sector                                     Recipient Name                          
I.1.a. Education, Level Unspecified        Afghanistan                  0.020691   
                                           Africa, regional             2.571572   
                                           America, regional            2.482967   
                                           Asia, regional               4.524518   
                                           Belarus                      0.015519   
...                                                                          ...   
VIII.3. Disaster Prevention & Preparedness Pakistan                     0.758688   
                                           Philippines                  0.187683   
                                           Somalia                      0.758685   
                                           West Bank and Gaza Strip     0.386239   
                                           Yemen                        0.178599   

                                                                     clim_rel_amount_2013  \
Sector                                     Recipient Name                                   
I.1.a. Education, Level Unspecified        Afghanistan                           0.000000   
                                           Africa, regional                      0.000000   
                                           America, regional                     0.000000   
                                           Asia, regional                        2.262259   
                                           Belarus                               0.000000   
...                                                                                   ...   
VIII.3. Disaster Prevention & Preparedness Pakistan                              0.758688   
                                           Philippines                           0.090172   
                                           Somalia                               0.758685   
                                           West Bank and Gaza Strip              0.193119   
                                           Yemen                                 0.000000   

                                                                     clim_adapt_amount_2013  \
Sector                                     Recipient Name                                     
I.1.a. Education, Level Unspecified        Afghanistan                             0.000000   
                                           Africa, regional                        0.000000   
                                           America, regional                       0.000000   
                                           Asia, regional                          2.262259   
                                           Belarus                                 0.000000   
...                                                                                     ...   
VIII.3. Disaster Prevention & Preparedness Pakistan                                0.379344   
                                           Philippines                             0.090172   
                                           Somalia                                 0.758685   
                                           West Bank and Gaza Strip                0.193119   
                                           Yemen                                   0.000000   

                                                                     clim_miti_amount_2013  \
Sector                                     Recipient Name                                    
I.1.a. Education, Level Unspecified        Afghanistan                            0.000000   
                                           Africa, regional                       0.000000   
                                           America, regional                      0.000000   
                                           Asia, regional   

In [252]:
df_merged = reduce(lambda left, right: pd.merge(left, right, on=['Sector', 'Recipient Name'], how='outer'), processed_grouped_dfs)


In [255]:
df_merged.to_csv('../upload_data/country_sector_analysis.csv')

df_merged = pd.read_csv('../upload_data/country_sector_analysis.csv')

In [257]:
year = 2013
country = 'Afghanistan'

selected_year = [col for col in df_merged.columns if col.endswith(f'{year}')]
selected_year.append('Sector')
df_merged = df_merged[df_merged['Recipient Name'] == country]
filtered_year_df = df_merged[selected_year]

In [258]:
filtered_year_df

,amount_2013,clim_rel_amount_2013,clim_adapt_amount_2013,clim_miti_amount_2013,non_miti_adapt_2013,non_clim_2013,Sector
0,0.020691,0.000000,0.000000,0.000000,0.020691,0.020691,"I.1.a. Education, Level Unspecified"
48,35.795844,0.000000,0.000000,0.000000,35.795844,35.795844,I.1.b. Basic Education
116,22.151253,1.888354,0.944177,0.944177,20.262899,20.262899,I.1.c. Secondary Education
221,1.293674,0.000000,0.000000,0.000000,1.293674,1.293674,"I.2.a. Health, General"
267,0.904807,0.000000,0.000000,0.000000,0.904807,0.904807,I.2.b. Basic Health
322,0.693010,0.000000,0.000000,0.000000,0.693010,0.693010,I.3. Population Policies/Programmes & Reproduc...
383,22.726831,3.212090,3.212090,0.000000,19.514740,19.514740,I.4. Water Supply & Sanitation
456,54.185554,0.000000,0.000000,0.000000,54.185554,54.185554,I.5.a. Government & Civil Society-general
562,13.003176,0.000000,0.000000,0.000000,13.003176,13.003176,"I.5.b. Conflict, Peace & Security"
687,8.093760,0.000000,0.000000,0.000000,8.093760,8.093760,II.1. Transport & Storage


In [9]:
dictionary_df['2015'][['Sector', 'SECTOR']]

,Sector,SECTOR
0,IV.1. General Environment Protection,41010
1,IV.1. General Environment Protection,41020
2,IV.2. Other Multisector,43010
3,IV.2. Other Multisector,43010
4,IV.2. Other Multisector,43030
...,...,...
12596,I.1.b. Basic Education,11230
12598,IV.2. Other Multisector,43010
12601,IV.2. Other Multisector,43010
12611,I.5.a. Government & Civil Society-general,15110
